# Setup del proyecto

En esta sección preparamos el entorno de trabajo necesario para el proyecto.

## Instalaciones

In [1]:
%pip install python-dotenv pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## importaciones

In [4]:
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path
import json
import os 
import pprint
import sqlite3

## Crear la BBDD

In [ ]:
import sqlite3

def generate_sqlite_database(sql_script, sqlite_db):

    # 1. Connect to the SQLite database
    conn = sqlite3.connect(sqlite_db)

    try:
        # 2. Read the SQL script from a file
        with open(sql_script, 'r') as f:
            sql_script = f.read()

        # 3. Execute the SQL script
        conn.executescript(sql_script)
        print("SQL script executed successfully.")

        # 4. Commit changes
        conn.commit()

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        # Rollback changes in case of an error
        conn.rollback()

    finally:
        # 5. Close the connection
        conn.close()

generate_sqlite_database('./data/chinook-sqlite.sql', './data/chinook.db')  